In [1]:
import logging
import math,random
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

from models.Attention import Attention
from models.Embeddings.BERT import BERT_Embedding
from models.Embeddings.GLOVE import GLOVE_Embedding
from models.Encoders.CNN import CNN_Encoder
from models.Encoders.RNN import RNN_User_Encoder
from models.Interactors.CNN import CNN_Interactor
from models.Interactors.FIM import FIM_Interactor
from models.Interactors.KNRM import KNRM_Interactor
from models.Interactors.BERT import BERT_Interactor

from models.ESM import ESM
from models.base_model import BaseModel

from data.configs.demo import config
from utils.utils import prepare

from models.base_model import BaseModel
from models.Modules.DRM import DRM_Matching
# from models.Modules.TFM import TFM

In [2]:
config.npratio = 1
config.batch_size = 2
config.his_size = 2
config.k = 3
config.embedding = 'bert'
config.bert = 'bert-base-uncased'
config.hidden_dim = 768

vocab, loaders = prepare(config, path="C:/MIND", cache="data/.vector_cache")
record = next(iter(loaders[0]))

[2021-08-03 19:51:32,732] INFO (root) Hyper Parameters are scale:demo, mode:train, batch_size:2, title_size:20, abs_size:40, his_size:2, learning_rate:0.001, vert_num:18, subvert_num:293, npratio:1, dropout_p:0.2, embedding_dim:300, hidden_dim:768, query_dim:200, head_num:16, epochs:8, metrics:auc,mean_mrr,ndcg@5,ndcg@10, device:cpu, k:3, step:[0], interval:10, val_freq:2, schedule:None, embedding:bert, bert:bert-base-uncased
[2021-08-03 19:51:32,733] INFO (root) preparing dataset...


In [5]:
embedding = BERT_Embedding(config)
encoderN = CNN_Encoder(config)
encoderU = RNN_User_Encoder(encoderN.hidden_dim)
docReducer = DRM_Matching(config.k)
# termFuser = TFM(config.his_size, config.k)
# interactor = CNN_Interactor(config)
interactor = BERT_Interactor(config)

esm = ESM(config, embedding, encoderN, encoderU, docReducer, None, interactor).to(config.device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.de

In [6]:
esm(record)

tensor([[-0.6933, -0.6930],
        [-0.6905, -0.6958]], grad_fn=<LogSoftmaxBackward>)

In [8]:
config.epochs = 8
config.val_freq = 2
esm.tune(config, loaders)

[2021-08-03 19:54:22,571] INFO (models.base_model) training...


AttributeError: type object 'config' has no attribute 'spadam'

torch.Size([2, 2, 512, 1, 768]) torch.Size([2, 2, 768])
